---
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Investigación Operativa I4051__\
__Autor: Rodrigo Maranzana__, Rmaranzana@frba.utn.edu.ar

---

# Algoritmo de Método de Camino Crítico (CPM)

Esta implementacion del algoritmo CPM tiene un objetivo didáctico y asegura una óptima performance de implementación.

Los pasos para el cálculo son los siguientes.

- Dados datos de: matriz de sucesores, predecesores, tiempos de tareas.
- Cálculo de fechas tempranas de proyecto.
- Cálculo de fechas tardías de proyecto.
- Cálculo de margen total y tareas críticas.

Los cálculos de fechas tempranas y tardías son similares. El primero calcula las fechas de adelante hacia atrás y el segundo de atrás hacia adelante.

Ambos utilizan la misma función de exploración de nodos. Esta función es útil para saber cuál puede ser el próximo nodo a ser calculado.

In [1]:
# Importamos librerías necesarias.
import numpy as np
from itertools import cycle
import pandas as pd

## Exploración de nodos en un grafo de proyectos

Creamos una función para encontrar nodos próximos a explorar. 

La siguiente función nos va a servir tanto para fechas tempranas como tardías. Ya que las condiciones de vecinos, predecesores o sucesores son las mismas.

En cada iteración, el próximo nodo tiene que cumplir:
- Tener todos sus vecinos explorados (predecesores en caso de fecha temprana y sucesores en caso de fecha tardía).
- Ser un nodo no explorado.

In [2]:
# Cálculo de próximo nodo a explorar.
# El input lista_validacion puede ser:
# - predecesores: si estoy calculando fecha temprana.
# - sucesores: si estoy calculando fecha tardia.

def proximo_nodo_a_explorar(lista_validacion, nodos_faltantes):
    
    # Creamos un ciclo de nodos faltantes.
    iter_nodos_faltantes = iter(nodos_faltantes)

    # Inicializamos validacion de nodo.
    nodo_apto = False
    
    # Mientras el nodo no sea apto a completar (validación falsa) continuamos.
    while nodo_apto == False:
    
        # Próximo nodo faltante.
        nodo_actual = next(iter_nodos_faltantes)

        # Para cada vecino del actual.
        for v in lista_validacion[nodo_actual]:

            # Si p esta en nodos faltantes.
            if v in nodos_faltantes:
                
                # El nodo no es apto, validar false y cortar ciclo.
                nodo_apto = False

                break
                
            else:
                
                # Por ahora, el nodo es apto.
                nodo_apto = True
    
    return nodo_actual

## Cálculo de fechas tempranas

Creamos la función que calcula la fecha temprana como:

$Ft_j = max(Ft_i + D_{ij})$

In [3]:
# Cálculo de fecha temprana.
def calcular_fecha_temprana(predecesores, tiempos, nodo_actual, fecha_temp):

    # Inicializamos tiempo candidato a temprano.
    fecha_temp_actual = 0
    
    # Para cada predecesor "P" del nodo actual:
    for p in predecesores[nodo_actual]:

        # Calculo de fecha temprana actual.
        candidato = fecha_temp[p] + tiempos[p, nodo_actual]
        
        # Comparar la fecha temprana nueva con los predecesores calculados.
        # Quedarse con la mayor.
        fecha_temp_actual = max(fecha_temp_actual, candidato)

    return fecha_temp_actual

Ciclo principal de cálculo de fecha temprana:

1) Exploración: buscamos el próximo nodo a completar mediante la función *proximo_nodo_a_explorar(.)*

2) Calculamos la fecha temprana del nodo encontrado con *calcular_fecha_temprana(.)*

3) Removemos el nodo de la lista de no explorados.

In [4]:
def calcular_proyecto_fechas_tempranas(nodo_inicial, predecesores, tiempos):

    # Inicializacion de lista de fechas tempranas.
    fecha_temp = [None] * tiempos.shape[0]
    fecha_temp[nodo_inicial] = 0

    # Lista de nodos faltantes y explorados.
    nodos_faltantes = list(range(1, tiempos.shape[0]))

    # Mientras no haya más nodos para explorar.
    while len(nodos_faltantes) != 0:

        # Buscamos el próximo nodo a calcular la fecha temprana.
        nodo_actual = proximo_nodo_a_explorar(predecesores, nodos_faltantes)

        # Calculamos fecha temprana del nodo actual y la escribimos en la lista.
        fecha_temp[nodo_actual] = calcular_fecha_temprana(predecesores, tiempos, nodo_actual, fecha_temp)

        # Removemos al nodo explorado de los faltantes.
        nodos_faltantes.remove(nodo_actual)

    return fecha_temp

## Cálculo de fechas tardías

Creamos la función que calcula la fecha tardía como:

$FT_i = min(FT_j - D_{ij})$

In [5]:
# Cálculo de fecha tardía.
def calcular_fecha_tardia(sucesores, tiempos, nodo_actual, fecha_tard):

    # Inicializamos tiempo candidato a tardío.
    fecha_tard_actual = 999
    
    # Para cada sucesor "s" del nodo actual:
    for s in sucesores[nodo_actual]:
    
        # Calculo de fecha temprana actual.
        candidato = fecha_tard[s] - tiempos[nodo_actual, s]
        
        # Comparar la fecha tardía nueva con los sucesores calculados.
        # Quedarse con la menor.
        fecha_tard_actual = min(fecha_tard_actual, candidato)

    return fecha_tard_actual

Ciclo principal de cálculo de fecha tardía:

1) Exploración: buscamos el próximo nodo a completar mediante la función *proximo_nodo_a_explorar(.)*

2) Calculamos la fecha tardía del nodo encontrado con *calcular_fecha_tardia(.)*

3) Removemos el nodo de la lista de no explorados.

In [6]:
def calcular_proyecto_fechas_tardias(nodo_final, fin_temprana, sucesores, tiempos):

    # Inicializacion de lista de fechas tardías.
    fecha_tard = [None] * tiempos.shape[0]
    fecha_tard[nodo_final] = fin_temprana

    # Lista de nodos faltantes y explorados.
    nodos_faltantes = list(range(0, tiempos.shape[0]))
    nodos_faltantes.remove(nodo_final)

    # Mientras no haya más nodos para explorar.
    while len(nodos_faltantes) != 0:

        # Buscamos el próximo nodo a calcular la fecha tardía.
        nodo_actual = proximo_nodo_a_explorar(sucesores, nodos_faltantes)

        # Calculamos fecha tardía del nodo actual y la escribimos en la lista.
        fecha_tard[nodo_actual] = calcular_fecha_tardia(sucesores, tiempos, nodo_actual, fecha_tard)

        # Removemos al nodo explorado de los faltantes.
        nodos_faltantes.remove(nodo_actual)
        
    return fecha_tard

## Ejemplo Ejercicio 2
Si bien podemos partir de una única matriz o lista de predecesores o sucesores y calcular el resto; con el objetivo de centrarnos únicamente en el algoritmo CPM, vamos a recibir como dato:
- Lista de predecesores: Es una lista, en donde cada índice es el nodo final y los valores, listas secundarias con los nodos iniciales.
- Lista de sucesores: Es una lista, en donde cada índice es el nodo inicial y los valores, listas secundarias con los nodos finales.
- Matriz de tiempos: Es una matriz Nodo-Nodo que indica los tiempos de cada tarea.

En este ejemplo, vamos a resolver el ejercicio 2 de la guía.

In [7]:
# Datos.
predecesores = [
    [],
    [0],
    [0, 1, 3],
    [1],
    [2, 3, 5],
    [3]
]

sucesores = [
    [1, 2],
    [2, 3],
    [4],
    [2, 4, 5],
    [],
    [4]
]

tiempos = np.array([
    [None,    2,    4, None, None, None],
    [None, None,    1,    4, None, None],
    [None, None, None, None,    3, None],
    [None, None,    2, None,    3,    2],
    [None, None, None, None, None, None],
    [None, None, None, None,    2, None]
])

# Nodo inicial y final.
nodo_inicial = 0
nodo_final = 4

Ejecutamos en orden los cálculos de fechas tempranas y tardías.

In [9]:
# Cálculo de fechas tempranas:
fechas_tempranas = calcular_proyecto_fechas_tempranas(nodo_inicial, predecesores, tiempos)

# Cálculo de fechas tardías:
fin_temprana = fechas_tempranas[nodo_final]

fechas_tardias = calcular_proyecto_fechas_tardias(nodo_final, fin_temprana, sucesores, tiempos)

Organizamos el output en un dataframe de pandas corriendo el índice +1 para coincidir con el enunciado.

In [10]:
# Nombres de los nodos +1 para coincidir con la guía.
nombres_nodos = range(1, tiempos.shape[0] + 1)

# Creamos un dataframe con las fechas.
df = pd.DataFrame({"Nodos": nombres_nodos, "Ft": fechas_tempranas, "FT": fechas_tardias})

# Imprimimos.
df

,Nodos,Ft,FT
0,1,0,0
1,2,2,2
2,3,8,8
3,4,6,6
4,5,11,11
5,6,8,9


## Cálculo de camino crítico

Iteramos sobre la lista de sucesores para encontrar cada uan de las tareas y calculamos el márgen total de cada tarea *i-j*:

$Mtot_{ij} = FT_j - Ft_i - D_{ij}$

In [11]:
# Inicializamos una lista de tareas.
tareas = []

# Iteramos para cada nodo actual i, su lista de sucesores js.
for i, js in enumerate(sucesores):
    
    # Para cada sucesor j de la lista.
    for j in js:
        
        # Calculamos el Margen total:
        margen_total_ij = fechas_tardias[j] - fechas_tempranas[i] - tiempos[i, j]
        
        # Calculamos la criticidad de la tarea:
        if margen_total_ij == 0:
            criticidad = "C"
        else:
            criticidad = " "
        
        # Agregamos la tarea a la lista.
        tareas.append((f"{i + 1}-{j + 1}", margen_total_ij, criticidad))
        
# Organizamos la información en un pandas dataframe.
dfm = pd.DataFrame(tareas, columns=["Tarea", "Margen total", "Criticidad"])
    
# Imprimimos.
dfm

,Tarea,Margen total,Criticidad
0,1-2,0,C
1,1-3,4,
2,2-3,5,
3,2-4,0,C
4,3-5,0,C
5,4-3,0,C
6,4-5,2,
7,4-6,1,
8,6-5,1,


## Adicional:
- ¿Cómo podemos calcular automáticamente una lista de sucesores o predecesores dada cualquiera de las matrices o listas que representan un grafo?
- ¿Cómo podemos calcular automáticamente el nodo inicial y final de una red para usarlo en las condiciones de borde (fecha temprana 0 en nodo inicial y fecha tardía igual a temprana en nodo final)?